<a href="https://colab.research.google.com/github/DaryaJavadi/data_science/blob/main/projects/Samed_vurgun_style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

semed_vurgun_path = "/content/drive/MyDrive/Semed_Vurgun_seirleri.txt"
with open(semed_vurgun_path) as f:
    semed_vurgun_text = f.read()

In [ ]:
semed_vurgun_text[:30]

'\nCAVANLARA XİTAB\nMənim düşkün '

In [ ]:
''.join(sorted(set(semed_vurgun_text.lower())))

'\x01\n !"\'()*,-./0123456789:;<>?[\\]^abcdefghijklmnopqrstuvwxyz§°»äçöüğışə̇–—‘’“”†‡…★'

In [ ]:
import unicodedata

def replace_combining_dot(text):
    text_nfd = unicodedata.normalize('NFD', text)

    text_without_dot = text_nfd.replace('\u0307', '')

    final_text = unicodedata.normalize('NFC', text_without_dot)

    return final_text

with open("/content/drive/MyDrive/Semed_Vurgun_seirleri.txt", 'r', encoding='utf-8') as file:
    semed_vurgun_text = file.read()

cleaned_text = replace_combining_dot(semed_vurgun_text)

with open("/content/drive/MyDrive/Semed_Vurgun_seirleri.txt", 'w', encoding='utf-8') as file:
    file.write(cleaned_text)

In [ ]:
''.join(sorted(set(semed_vurgun_text.lower())))

'\x01\n !"\'()*,-./0123456789:;<>?[\\]^abcdefghijklmnopqrstuvwxyz§°»äçöüğışə̇–—‘’“”†‡…★'

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character',
                                                   standardize = 'lower')

text_vec_layer.adapt(semed_vurgun_text)
encoded = text_vec_layer([semed_vurgun_text][0])

In [ ]:
encoded -= 2
vocab_size = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [ ]:
dataset_size

840401

In [ ]:
vocab_size

86

In [ ]:
def to_dataset(sequence, length, seed = None, shuffle = False, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window: window.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [ ]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length = length, shuffle = True,
                       seed = 42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length = length)
test_set = to_dataset(encoded[1_060_000:], length = length)

In [ ]:
import tensorflow as tf

tf.random.set_seed(42)


model= tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = 16),
    tf.keras.layers.GRU(128, return_sequences= True),
    tf.keras.layers.Dense(vocab_size, activation="softmax")

])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer= tf.keras.optimizers.Nadam(learning_rate=1e-3),
              metrics=['accuracy'])

model_ckpt=tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model.keras",monitor="val_accuracy",save_best_only=True
)

model.fit(train_set,
          validation_data=valid_set,
          epochs=1,
          callbacks=[model_ckpt])

  26260/Unknown 328s 12ms/step - accuracy: 0.5173 - loss: 1.6837

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


26260/26260 ━━━━━━━━━━━━━━━━━━━━ 328s 12ms/step - accuracy: 0.5173 - loss: 1.6837


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


In [ ]:
semed_vurgun_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [ ]:
def next_char(text, temperature = 1):
    text = tf.constant([text])
    y_proba = semed_vurgun_model.predict(text)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extent_text(text, chars = 50, temperature = 1):
    for _ in range(chars):
        text +=next_char(text, temperature)
    return text

In [ ]:
extent_text('Bu tutduğu işdən', chars = 100, temperature = 0.1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━

'Bu tutduğu işdən bu dəfə\nbu də qardaşlar bir aygün tək.\ndərd elyar də deyirəm bu dəfə bu dərd sevindir...\n- gözündən'